<a href="https://colab.research.google.com/github/WebiksInc/Stop-Words-Hebrew/blob/main/prepare_stop_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# List Stop Words in Hebrew

In [1]:
# Install pyconll to handle UD
try:
    import pyconll
except ImportError as e:
    !pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import package
import numpy as np
import pandas as pd
import requests as req
import pyconll

In [3]:
# URL data train/test/dev
ud_dev = "https://raw.githubusercontent.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/master/he_iahltwiki-ud-dev.conllu"
ud_test = "https://github.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/raw/master/he_iahltwiki-ud-test.conllu"
ud_train = "https://github.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/blob/master/he_iahltwiki-ud-train.conllu?raw=true"

In [4]:
# download data
def get_data(url: str, name: str):
    res = req.get(url)
    file = open(name + ".txt", "w")
    file.write(res.text)
    file.close()


get_data(ud_dev, "ud_dev")
get_data(ud_test, "ud_test")
get_data(ud_train, "ud_train")

In [5]:
# load the UD data
dev = pyconll.load_from_file("/content/ud_dev.txt")
train = pyconll.load_from_file("/content/ud_test.txt")
test = pyconll.load_from_file("/content/ud_train.txt")

In [6]:
# Word extraction from the UD. Cleaning and adding words
def remove_niqqud_from_string(my_string):
    return "".join(["" if 1456 <= ord(c) <= 1479 else c for c in my_string])


def Extracts_stopwords(dat: pyconll):
    # Extracts the stopwords form the UD Schema
    POS_tag = ["DET", "ADP", "PRON", "CCONJ", "SCONJ", "SYM"]
    lst_words = [
        token.form for sentence in dat for token in sentence if token.upos in POS_tag
    ]
    return lst_words


words_plus = {
    "אגב",
    "הואיל",
    "הלא",
    "הרי",
    "כיאלו",
    "כיון",
    "יען",
    "על" "מנת",
    "ודאי",
    "גם",
    "אפילו",
    "יש",
    "רק",
}
words_minus = {
    "%",
    "&",
    "(",
    "+",
    "-",
    "/",
    ":",
    "=",
    "–",
    "°",
    "בינ",
    "שורת",
    "אודות",
    "לאזור",
    "במקביל",
    "כשב-1948",
    "ד",
    "בתוכ",
    "הו",
    "י",
    "ך",
    "ם",
    "ן",
    "נ'",
    "נאומ",
}

# Extracts the stopwords form the UD Schema
stopwords_raw = set(
    Extracts_stopwords(dev) + Extracts_stopwords(train) + Extracts_stopwords(test)
)
# Optionally, delete the words with the niqqud
# stopwords_raw = set(map(remove_niqqud_from_string,stopwords_raw))
# Delete incorrect words
stopwords = stopwords_raw - words_minus
# Add missing words that we identified . Note that there is another addition of words below
stopwords = stopwords | words_plus

# Note: The list includes English words that happened to be in the database. For a complete list in English we recommend nltk package.

In [7]:
# Add prepositions. A detailed explanation of the need appears in README

al = [
    "אלי",
    "אליך",
    "אליך",
    "אליו",
    "אליה",
    "אלינו",
    "אליכם",
    "אליכן",
    "אליהם",
    "אליהן",
]
hal = [
    "עלי",
    "עליך",
    "עליך",
    "עליו",
    "עליה",
    "עלינו",
    "עליכם",
    "עליכן",
    "עליהם",
    "עליהן",
]
him = [
    "עמי",
    "עמך",
    "עמו",
    "עמה",
    "עמנו",
    "עמכם",
    "עמכן",
    "עמם",
    "עמן",
    "עמהם",
    "עמהן"
]
ath_with = [
    "אתן",
    "אתם",
    "אתכן",
    "אתכם",
    "אתנו",
    "אתה",
    "אתו",
    "אתך",
    "אתי"
]
ath = [
    "אותי",
    "אותך",
    "אותו",
    "אותה",
    "אותנו",
    "אתכם",
    "אתכן",
    "אותם",
    "אותן"
]
shel = [
    "שלי",
    "שלך",
    "שלו",
    "שלה",
    "שלנו",
    "שלכם",
    "שלכן",
    "שלהם",
    "שלהן",
]
men = [
    "ממני",
    "ממך",
    "ממנו",
    "ממנה",
    "מכם",
    "מכן",
    "מהם",
    "מהן"
]
azhel = [
    "אצלי",
    "אצלך",
    "אצלו",
    "אצלה",
    "אצלנו",
    "אצלכם",
    "אצלכן",
    "אצלם", 
    "אצלן"
]
bshvil = [
    "בשבילי",
    "בשבילך",
    "בשבילו",
    "בשבילה",
    "בשבילנו",
    "בשבילכם",
    "בשבילכן",
    "בשבילם",
    "בשבילן",
]
bin = [
    "ביני",
    "בינך",
    "בינו",
    "בינה",
    "בינינו",
    "ביניכם",
    "ביניכן",
    "ביניהם",
    "ביניהן",
    "בינם",
    "בינן",
]
lafi = [
    "לפי",
    "לפיך",
    "לפיו",
    "לפיה",
    "לפינו",
    "לפיכם",
    "לפיכן",
    "לפיהם",
    "לפיהן"
]
lafni = [
    "לפני",
    "לפניך",
    "לפניו",
    "לפניה",
    "לפנינו",
    "לפניכם",
    "לפניכן",
    "לפניהם",
    "לפניהן",
]
chmo = [
    "כמוני",
    "כמוך",
    "כמוהו",
    "כמוה",
    "כמונו",
    "כמוכם",
    "כמוכן",
    "כמוהם",
    "כמוהן"
]
lakrt = [
    "לקראתי",
    "לקראתך",
    "לקראתו",
    "לקראתה",
    "לקראתנו",
    "לקראתכם",
    "לקראתכן",
    "לקראתם",
    "לקראתן",
]
pepositions = {
    "אל": al,
    "על": hal,
    "עם": him,
    "(=עם) את": him,
    "את (תוויות מושא ישיר": ath,
    "של": shel,
    "מן": men,
    "אצל": azhel,
    "בשביל": bshvil,
    "בין": bin,
    "לפי": lafi,
    "לפני": lafni,
    "כמו": chmo,
    "לקראת": lakrt,
}
pepositions_lst = [item for sublist in list(pepositions.values()) for item in sublist]

stopwords = stopwords | set(pepositions_lst)
print(
    ", ".join(
        [
            word if index % 15 != 0 else "\n," + word
            for index, word in enumerate(stopwords)
        ]
    )
)


,מתוכן, טרם, עלינו, עליכם, עלמנת, כיוון, עמ, בעבור, מחצית, ערכ, אצלנו, רוב, ודאי, הָ, מי, 
,מ, ביניהם, of, אצל, אצלי, ב, ני, יו, בינם, the, לפניהם, אותם, משל, You, לפיו, 
,ממנו, מרבית, אצלם, אליכם, בתור, לפינו, לציד, בינו, לגבי, כלשהם, אל, עצמה, עמה, לא, ממני, 
,אלמלא, במקומ, מטעם, עובר, בהתחשב, מעין, כעבור, היא, בין, אותי, בעקבות, מלבד, שלכן, למן, כמוה, 
,היות, בפני, ל, זה, כמוך, תוך, הלא, זהו, עצמך, עליה, הודות, כ, עליהם, בטרם, מציד, 
,ביני, אלינו, אֶל, בזכות, לפיהם, לבין, לכבוד, כמונו, כאשר, to, אצלו, עבר, כמות, וַ, החל, 
,לאורך, הּ, מאז, כלשהו, רק, עמו, ְהַ, ספר, מה, בהתבסס, עד, שכן, אליך, איזה, לפיכם, 
,אותך, בשבילך, בעוד, כולו, מבחינת, את, ע"פ, לידי, מעט, לפנות, עמן, a, מאשר, מפני, Your, 
,לפניכן, מול, לתוך, לפיכן, כמוהו, שמא, בינך, לולא, כלומר, מפאת, אם, כִּי, זאת, הַ, אותה, 
,ככל, לאחרי, אף, לקראתו, ה, עמכם, אבל, אצלך, לקראתי, בתוך, בשבילה, לקראתך, הללו, פי, לפיך, 
,לעבר, אוּלָם, מהן, מחמת, חרף, דרך, זו, אנו, לפני, אליו, בגדר, לקראתכם, דוגמת, אי, כיאלו, 
,כנגד, אית, שלה, אולם,

In [8]:
# Download the list
from google.colab import files

pd.DataFrame({"stopswords": list(stopwords)}).to_csv(
    "stopswords_list.txt", index=False, sep="\t"
)
files.download("stopswords_list.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>